In [2]:
import matplotlib.pyplot as plt
import numpy as np

In [3]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models

# Load CIFAR-10 dataset
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

In [3]:
print(train_images.shape)
print(train_labels.shape)
print(test_images.shape)
print(test_labels.shape)

(50000, 32, 32, 3)
(50000, 1)
(10000, 32, 32, 3)
(10000, 1)


In [4]:
# Define ResNet model
def build_resnet(input_shape, num_classes):
    inputs = tf.keras.Input(shape=input_shape)
    x = layers.Conv2D(64, (3, 3), padding='same', activation='relu')(inputs)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)

    # Residual blocks
    num_res_blocks = 3
    for _ in range(num_res_blocks):
        residual = x

        x = layers.Conv2D(64, (3, 3), padding='same', activation='relu')(x)
        x = layers.Conv2D(64, (3, 3), padding='same')(x)
        x = layers.BatchNormalization()(x)
        x = layers.add([x, residual])
        x = layers.Activation('relu')(x)

    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(num_classes, activation='softmax')(x)

    model = tf.keras.Model(inputs=inputs, outputs=x)
    return model

In [5]:
# Build ResNet model
input_shape = train_images.shape[1:]
num_classes = 10
resnet_model = build_resnet(input_shape, num_classes)

In [6]:
# Compile the model
resnet_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [7]:
# Train the model
resnet_model.fit(train_images, train_labels, epochs=10, validation_data=(test_images, test_labels))

Epoch 1/10
1563/1563 [==============================] - 496s 316ms/step - loss: 1.3693 - accuracy: 0.5034 - val_loss: 1.8259 - val_accuracy: 0.3933
Epoch 2/10
1563/1563 [==============================] - 515s 329ms/step - loss: 1.0023 - accuracy: 0.6465 - val_loss: 1.4614 - val_accuracy: 0.5263
Epoch 3/10
1563/1563 [==============================] - 513s 328ms/step - loss: 0.8599 - accuracy: 0.6986 - val_loss: 1.1115 - val_accuracy: 0.6110
Epoch 4/10
1563/1563 [==============================] - 514s 329ms/step - loss: 0.7529 - accuracy: 0.7389 - val_loss: 0.9889 - val_accuracy: 0.6675
Epoch 5/10
1563/1563 [==============================] - 511s 327ms/step - loss: 0.6791 - accuracy: 0.7642 - val_loss: 0.9265 - val_accuracy: 0.6781
Epoch 6/10
1563/1563 [==============================] - 513s 329ms/step - loss: 0.6221 - accuracy: 0.7848 - val_loss: 0.9541 - val_accuracy: 0.6825
Epoch 7/10
1563/1563 [==============================] - 515s 329ms/step - loss: 0.5675 - accuracy: 0.8042 - val_

In [8]:
# Evaluate the model
test_loss, test_acc = resnet_model.evaluate(test_images, test_labels)
print("Test accuracy:", test_acc)

313/313 [==============================] - 26s 84ms/step - loss: 0.8112 - accuracy: 0.7332
Test accuracy: 0.7332000136375427


In [16]:
resnet_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 32, 32, 3)]          0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 32, 32, 64)           1792      ['input_1[0][0]']             
                                                                                                  
 max_pooling2d (MaxPooling2  (None, 16, 16, 64)           0         ['conv2d[0][0]']              
 D)                                                                                               
                                                                                                  
 conv2d_1 (Conv2D)           (None, 16, 16, 64)           36928     ['max_pooling2d[0][0]']   